## Research Agent
A research agent is a Generative AI–powered system built to independently perform research on a defined topic by emulating how a human researcher reasons and operates.
Instead of merely retrieving information, a research agent interprets the research objective, plans the methodology, gathers relevant data, and synthesizes insights
into a structured output.

#### Research Agent orchestrates four sub agents:
#### 1) Scoping - Uses human interaction to define objectives and constraints for the research topic
#### 2) Briefing - Defines scope brief for the research using human interaction based scope definition
#### 3) Execution - Performs research on topic based on scope brief and generates research content.
#### 4) Drafting - Drafts research report based on research content

#### In this example, we are using linear deterministic execution of the scoping, briefinf, execution and drafting sub agents.

<img src="./images/Linear.png" alt="Sample" width="500"/>


## Setup
#### Configure Open API Key as Open API Model is used in this example
#### Configure Tavily API Key as Tavily is used for internet search tool with the agent
#### Using Strands SDK for agent configuration and multi-agent orchestration

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = 'YOUR_OEPN_API_KEY'
os.environ["TAVILY_API_KEY"] ='YOUR_TAVILY_API_KEY'

In [ ]:
from strands import Agent, tool
from strands.models.openai import OpenAIModel
import json

## 0) Initiate
### Provide a research topic
### Use GPT-5-Nano model. You can choose to use another reasoning model as well

In [ ]:
research_topic = "Adoption of Digital Coins in Banks"

In [ ]:
model = OpenAIModel(model_id="gpt-5-nano")

## 1) Create Scoping Agent
#### Used to define scope of the research using human input. Model generates questions for the topic which humans can respond to.
#### In real world example, you would like to allow humans to provide additional scoping details beyond the questions.
#### For this example, limited the questions to 5 only but feel free to change prompt to generate the kind of questions you want to generate.

In [ ]:
scoping_system_prompt = """
You are a clarification agent.

Your task is to take a single user question about a topic and generate a list of short, well-scoped follow-up questions that help gather
missing details, clarify intent, assumptions, constraints, or context.

Rules:
- Ask maximum 5 questions.
- Questions must be concise and specific.
- Avoid redundant questions.
- Focus on clarifying scope and goals.
- Ask about intendent audience
- Ask about size of the research in terms of words
- Do NOT answer the original question
- Do NOT include explanations, comments, or extra text

Output format:
- Return ONLY a valid string array

Example structure (do not include this example in output):

[
    "What is the target audience?",
    "Are there any constraints on time or budget?"
]


"""

scoping_agent = Agent(system_prompt=scoping_system_prompt, model=model)
response = scoping_agent(research_topic)
scope_questions = response.message["content"][0]["text"]

In [ ]:
# Prompt user to take input on scoping questions
research_scope = []

for q in json.loads(scope_questions):
    r = input(q)
    research_scope.append("Question: " + q + " and Answer: " + r)

In [ ]:
# Printing user input to scoping questions
print(research_scope)

## 2) Create Research Briefing, Execution and Drafing Agent
#### Briefing Agent - Generates brief for the research based on scope provided by scoping agent
#### Execution Agent - Performs research and generate report content based on brief provided by the briefing agent The Agent uses TAVILY API as tool to search over internet.
#### Drafting Agent - Drafts the report for the research content generated by execution agent

### 2.1 Briefing Agent

In [ ]:
brief_system_prompt = f"""
        You are a research-brief generation agent.
               
        Task:
        - Create research brief for a research topic based on answers to the research scoping questions provider.
        
        Guidelines:
        - Clearly state the research objective and purpose
        - Define in-scope and out-of-scope areas
        - Identify key themes, assumptions, and constraints from the questions and answers
        - Highlight target audience and intended use of the research
        - Propose high-level research questions or focus areas
        - Keep the brief concise, structured, and neutral
        - Do NOT add new assumptions beyond the provided inputs
        - Do NOT include explanations or meta commentary
        
        Output:
        - Return ONLY the research brief
        - Use clear headings and bullet points where appropriate
        - No preamble or closing text
        """
briefing_agent = Agent(system_prompt=brief_system_prompt, model=model)

### 2.2 Execution Agent

In [ ]:
from tavily import TavilyClient
from typing import Dict, Any
import datetime

CURRENT_DATE = datetime.datetime.now()

tavily_client = TavilyClient()

@tool
def web_search(query: str) -> str:
    """Search query over internet to fetch results
    Args:
        query: query which is searched over internet
    """
    response = tavily_client.search(query)
    output = ""
    count = 1
    for r in response["results"]:
        output = output +  "{0}- Content: {1}. Citation: {2} \n".format(count, r["content"], r["url"])
        count = count + 1
    return output

In [ ]:
execution_system_prompt = f"""
        You are a research execution agent which performs research based on reseach brief provided to generate search report.

        Current Date: 
        {CURRENT_DATE}
        
        Available Tools:
        - web_search: Use this tool to search the internet for up-to-date, reliable information relevant to the research topic and objectives.
      
        Task:
        - Conduct research strictly according to the research brief
        - Use the web_search tool to gather current, verifiable information
        - Prefer recent and authoritative sources, considering the current date
        - Synthesize findings into a clear, structured research output
        
        Rules:
        - Adhere exactly to the objectives, scope, exclusions, constraints, and assumptions in the research brief
        - Do NOT expand beyond the brief or introduce new goals
        - Do NOT speculate or add unsupported assumptions
        - Do NOT include methodology explanations or meta commentary
        - Do NOT restate the research brief unless explicitly required
        - Maintain a neutral, factual tonetools=[calculator, file_read, shell]
        
        Output:
        - Return ONLY the research report
        - Organize content using clear headings and bullet points
        - No preamble, explanations, or closing text

        """
execution_agent = Agent(system_prompt=execution_system_prompt, model=model, tools=[web_search])

### 2.3 Drafting Agent

In [ ]:
drafting_system_prompt = f"""
        You are a research report drafting agent which draft a research report based on report content provided.
        
        Task:
        - Write the research report in a clean, clear, and professional manner
        - Improve clarity, coherence, and readability while preserving original meaning
        - Organize the report with a logical structure, clear section headings, and well-formed paragraphs
        - Maintain an objective, academic or professional tone
        
        Rules:
        - Use ONLY the information present in the input research report
        - Do NOT add, remove, or infer new facts
        - Do NOT introduce new citations or sources
        - Do NOT modify the substance or conclusions of the report
        - Do NOT hallucinate or supplement missing information
        - Ensure all rewritten content is directly traceable to the original report
        
        Output Structure:
        1. Write Research Report  
           - Provide title in the begining and conclusion in the end
           - Structured with headings and paragraphs  
        2. Citations  
           - Present all citations separately as provided in the input
        
        Output:
        - Return ONLY the report report along with and citations
        - No explanations, meta commentary, or extra text
        """

drafting_agent = Agent(system_prompt=drafting_system_prompt, model=model)

## 3) Research Orchestration
#### In this example, we are using linear deterministic orchestration of the scoping, briefing, execution and drafting sub agents.
#### Scoping agent is already used earlier, you will see a graph orchestrator created where scoping agent outcome is input and orchestrator orchestrates briefing, execution and drafting in linear sequence

In [ ]:
from strands.multiagent import GraphBuilder

builder = GraphBuilder()

builder.add_node(briefing_agent, "briefing_agent")
builder.add_node(execution_agent, "execution_agent")
builder.add_node(drafting_agent, "drafting_agent")

builder.add_edge("briefing_agent", "execution_agent")
builder.add_edge("execution_agent", "drafting_agent")
builder.set_entry_point("briefing_agent")

builder.set_execution_timeout(300)   # 5 minute timeout

graph = builder.build()

result = graph("The research topic is {0} and scoping questions are {1}".format(research_topic,research_scope))

In [ ]:
# Final Report
print(result.results["drafting_agent"].result.message["content"][0]["text"])